<a href="https://colab.research.google.com/github/HarshithaaRG/Football-Match-Prediction/blob/main/Footballprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/sample_submission.csv
/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/X_test.csv
/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/y_train.csv
/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/X_train.csv


In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


X_train = pd.read_csv('/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/X_train.csv')
y_train = pd.read_csv('/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/y_train.csv')['result']


test_data = pd.read_csv('/kaggle/input/gdsc-vashisht-ml-hackathon-open-for-all/X_test.csv')


X_train['date'] = pd.to_datetime(X_train['date'])
test_data['date'] = pd.to_datetime(test_data['date'])


X_train['date'] = (X_train['date'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
test_data['date'] = (test_data['date'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')


X_train['country'] = X_train['country'].astype('category')
test_data['country'] = test_data['country'].astype('category')

categorical_features = ['home_team', 'away_team', 'city', 'tournament', 'neutral', 'country']
numerical_features = ['date']


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=100)


label_encoders = {}
for feature in categorical_features:
    label_encoder = LabelEncoder()
    X_train[feature] = X_train[feature].astype(str)
    X_val[feature] = X_val[feature].astype(str)
    test_data[feature] = test_data[feature].astype(str)
    label_encoder.fit(np.concatenate([X_train[feature], X_val[feature], test_data[feature]]))
    label_encoders[feature] = label_encoder


    unseen_labels = [label for label in test_data[feature].unique() if label not in label_encoder.classes_]
    label_encoder.classes_ = np.concatenate([label_encoder.classes_, unseen_labels])


    X_train[feature] = label_encoder.transform(X_train[feature])
    X_val[feature] = label_encoder.transform(X_val[feature])
    test_data[feature] = label_encoder.transform(test_data[feature])


cat_encoder = OneHotEncoder(handle_unknown='ignore')
X_train_cat = cat_encoder.fit_transform(X_train[categorical_features])
X_val_cat = cat_encoder.transform(X_val[categorical_features])
X_test_cat = cat_encoder.transform(test_data[categorical_features])


logistic_regression = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)


logistic_regression.fit(X_train_cat, y_train)


y_val_pred = logistic_regression.predict(X_val_cat)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation accuracy: {val_accuracy}')


y_test_pred = logistic_regression.predict(X_test_cat)


submission = pd.DataFrame({'row_id': range(len(y_test_pred)), 'result': y_test_pred})
submission.to_csv('submission.csv', index=False)

Validation accuracy: 0.5692510801728277
